In [14]:
import torch
import os
import torchaudio
from torch import nn
from torch.utils.data import DataLoader, TensorDataset

In [118]:
DATA_PATH = '../../data/music_genre_dataset'

WAV_PATH = os.path.join(DATA_PATH, 'genres_original')
TEST_PATH = os.path.join(DATA_PATH, 'test_set')

In [119]:
### Arguments
### path: path of either wav or png directory
### Returns
### dictionary of 'genre' -> number
def extract_classes(path=WAV_PATH):
    classes = {}
    i = 0
    for genre in os.listdir(path):
        classes[genre] = i
        i += 1
    return classes
CLASSES = extract_classes()
print(CLASSES)

{'blues': 0, 'classical': 1, 'country': 2, 'disco': 3, 'hiphop': 4, 'jazz': 5, 'metal': 6, 'pop': 7, 'reggae': 8, 'rock': 9}


The classification of genre is enumerated in the following way: {'blues': 0, 'classical': 1, 'country': 2, 'disco': 3, 'hiphop': 4, 'jazz': 5, 'metal': 6, 'pop': 7, 'reggae': 8, 'rock': 9}

In [120]:
def get_genre_lable_vector(genre):
    if genre == 'blues':
        return torch.tensor([[1,0,0,0,0,0,0,0,0,0]])
    elif genre == 'classical':
        return torch.tensor([[0,1,0,0,0,0,0,0,0,0]])
    elif genre == 'country':
        return torch.tensor([[0,0,1,0,0,0,0,0,0,0]])
    elif genre == 'disco':
        return torch.tensor([[0,0,0,1,0,0,0,0,0,0]])
    elif genre == 'hiphop':
        return torch.tensor([[0,0,0,0,1,0,0,0,0,0]])
    elif genre == 'jazz':
        return torch.tensor([[0,0,0,0,0,1,0,0,0,0]])
    elif genre == 'metal':
        return torch.tensor([[0,0,0,0,0,0,1,0,0,0]])
    elif genre == 'pop':
        return torch.tensor([[0,0,0,0,0,0,0,1,0,0]])
    elif genre == 'reggae':
        return torch.tensor([[0,0,0,0,0,0,0,0,1,0]])
    else:
        return torch.tensor([[0,0,0,0,0,0,0,0,0,1]])

In [121]:
'''
Returns a dataset with audio data and its classification

{
    'train': [[f0-fn],[f0-fn],...],
    'target': [1,3,...]
}
'''
def getTrainDataSet(path, duration=10, sr=22050):
    genres = os.listdir(path)
    n_frame = duration * sr        
    audio_tensor = torch.Tensor()
    label_tensor = torch.Tensor()
    for genre in genres:
        genre_path = os.path.join(path, genre)
        files = os.listdir(genre_path)
        for file in files:
            file_path = os.path.join(genre_path, file)
            audio_frames = torchaudio.load(file_path, num_frames=n_frame)
            audio_tensor = torch.cat((audio_tensor, audio_frames[0]), dim=0)            
            label_tensor = torch.cat((label_tensor, get_genre_lable_vector(genre)), dim=0)
    return audio_tensor, label_tensor

train, label = getTrainDataSet(WAV_PATH)
print(train.shape)
print(label.shape)
print(train)
print(label)

torch.Size([999, 220500])
torch.Size([999, 10])
tensor([[ 0.0073,  0.0166,  0.0076,  ...,  0.0762,  0.0768,  0.0813],
        [ 0.0034,  0.0043,  0.0014,  ...,  0.0382,  0.0512,  0.0578],
        [ 0.0190,  0.0477,  0.0294,  ...,  0.1489,  0.1865,  0.2213],
        ...,
        [-0.0341, -0.0384,  0.0031,  ...,  0.0347,  0.0670,  0.0928],
        [ 0.0859,  0.1448,  0.1429,  ...,  0.0078,  0.0145,  0.0208],
        [-0.0211, -0.0345, -0.0354,  ...,  0.0570,  0.0645,  0.0568]])
tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 1.],
        [0., 0., 0.,  ..., 0., 0., 1.],
        [0., 0., 0.,  ..., 0., 0., 1.]])


In [122]:
test, test_label = getTrainDataSet(TEST_PATH)

In [73]:
BATCH_SIZE = 64

In [123]:
train_tensor = TensorDataset(train, label)
test_tensor = TensorDataset(test, test_label)

train_loader = DataLoader(dataset = train_tensor, batch_size = BATCH_SIZE, shuffle = True)
test_loader = DataLoader(dataset = test_tensor, batch_size = BATCH_SIZE, shuffle = True)

In [124]:
input_layer_size = 0

for X, y in train_loader:
    input_layer_size = X.shape[1]
    print(X.shape[1])
    break

220500


In [125]:
torch.full(size=(10,), fill_value=1)

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [126]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()        
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_layer_size, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.Softmax()          
        )

    def forward(self, x):        
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)

Using cpu device


In [127]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [128]:
# Initialize the loss function
loss_fn = nn.MSELoss() # mean absolute error , mse

In [129]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [130]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)        
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad() # fit
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [131]:
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()            
            correct += (pred.argmax(1) == y.argmax(1)).type(torch.float).sum().item()

    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [132]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_loader, model, loss_fn, optimizer)
    test_loop(test_loader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------


c:\Users\amos.zuercher\.conda\envs\ai\Lib\site-packages\torch\nn\modules\module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


loss: 0.090152  [    0/  999]
Test Error: 
 Accuracy: 81.0%, Avg loss: 0.000959 

Epoch 2
-------------------------------
loss: 0.048763  [    0/  999]
Test Error: 
 Accuracy: 99.0%, Avg loss: 0.000198 

Epoch 3
-------------------------------
loss: 0.010134  [    0/  999]
Test Error: 
 Accuracy: 100.0%, Avg loss: 0.000031 

Epoch 4
-------------------------------
loss: 0.003681  [    0/  999]
Test Error: 
 Accuracy: 100.0%, Avg loss: 0.000016 

Epoch 5
-------------------------------
loss: 0.000271  [    0/  999]
Test Error: 
 Accuracy: 100.0%, Avg loss: 0.000011 

Done!


In [86]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [ ]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

In [ ]:
classes = [
    "blues",
    "classical",
    "country",
    "disco",
    "hiphop",
    "jazz",
    "metal",
    "pop",
    "reggae",
    "rock",
]

model.eval()
x, y = test_tensor[0][0], test_tensor[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')